In [ ]:
using Random
using Distributions
using DataFrames
using CSV

In [ ]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    sites = collect(1:N)
    dirs = ['x','y']
    interactions = DataFrame(["$site,$dir" => [] for site in sites for dir in dirs])    
    for site in 1:N
        up = mod(site-1-Lx,N)
        down = mod(site-1+Lx,N)
        right = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)+1,Lx))
        left = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)-1,Lx))
        push!(interactions[!,"$site,x"],[left+1,right+1])
        push!(interactions[!,"$site,y"],[up+1,down+1])
    end
    
    return interactions
end

In [ ]:
function rows_and_columns(Lx,Ly)
    
    N =Lx*Ly
    rows = DataFrame(["row$row" => [] for row in 1:Ly])    
    cols = DataFrame(["col$col" => [] for col in 1:Lx])
    for site in 1:N
        row = Int((((site-1) - ((site-1)%Lx))/Ly)+1)
        col = Int(((site-1) % Lx)+1)
        push!(rows[!,"row$row"],site)
        push!(cols[!,"col$col"],site)
    end
    
    return rows, cols
end

In [ ]:
function Calculate_XY_Energy(config,interactions)
    
    N = length(config)
    totalenergy = 0
    
    for site in (1:N)
        neighbors_x = interactions[!,"$site,x"][1,1]
        neighbors_y = interactions[!,"$site,y"][1,1]
        energy_contributions_x = 0.5*-1*cos.(config[site].-config[neighbors_x])
        energy_contributions_y = 0.5*-1*cos.(config[site].-config[neighbors_y])
        totalenergy += sum(energy_contributions_x) + sum(energy_contributions_y)
    end
    
    return totalenergy

end

In [ ]:
function XY_update(config,energy,interactions,beta,energy_func)
    
    random_angles =rand(Float64,length(config))*2*pi
#     new_config = mod2pi.(copy(config).+random_angles)
    new_energy = energy_func(random_angles,interactions)
    deltaE = new_energy-energy
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    delta_accept_count = 0
    
    if exp_beta_deltaE > rand_value
        config = random_angles
        delta_accept_count += 1
    else
        deltaE = 0.
    end
    
    return config, deltaE, delta_accept_count
end

In [ ]:
function Wolff_step(L::Int,neighbours, spins_angles,T)
    # angle defining flipping axis
    N = L^2
    α = rand(Uniform(0,2*pi))
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        spin_being_checked = spins_to_check[1]
        θ = spins_angles[spin_being_checked]
        all_neighbours = neighbours[spin_being_checked ,:]
        neigh_not_in_cluster = setdiff(all_neighbours,cluster)
        for i in neigh_not_in_cluster
            θ_neigh = spins_angles[i]
            prob = 1-exp(min(0,-(2/T)*cos(θ-α)*cos(θ_neigh-α)))
            # println((2/T)*cos(θ-α)*cos(θ_neigh-α))
            # println(prob)
            if rand() < prob
                append!(cluster, i)
                append!(spins_to_check, i)
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    for i in cluster
        spins_angles[i] = mod((π +2*α - spins_angles[i]),2*π)
    end
    return spins_angles
end

In [ ]:
function XY_cluster_update(config,energy,interactions,beta,energy_func) #(L::Int, neighbours, spins_angles, T) #
    # angle defining flipping axis
    N = length(config)
    alpha = rand(Uniform(0,2*pi))
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        site = spins_to_check[1]
        theta = config[site]
        neighborsx = interactions[!,"$site,x"][1,1]
        neighborsy = interactions[!,"$site,y"][1,1]
        neighbors = append!(copy(neighborsx),neighborsy)
        neighbors_not_in_cluster = setdiff(neighbors,cluster)
        for i in neighbors_not_in_cluster
            theta_neighbor = config[i]
            prob = 1-exp(min(0,-(2*beta)*cos(theta-alpha)*cos(theta_neighbor-alpha)))
            if prob > rand()
                append!(cluster, i)
                append!(spins_to_check, i)
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    
    for i in cluster
        config[i] = mod((pi +2*alpha - config[i]),2*pi)
    end
    
    new_energy = energy_func(config,interactions)
    deltaE = new_energy-energy
    delta_accept_count = 0
    
    return config, deltaE, delta_accept_count
end

In [ ]:
function spin_stiffness(beta,config,interactions,rows,cols)
    
    N = length(config)
    L = Int(sqrt(N)) # obv for a square lattice only
    
    avg_ps_over_rows = 0
    for row in 1:L
        sites_in_row = rows[!,"row$row"]
        cos_contributions_x = 0
        sin_contributions_x = 0
        for site in sites_in_row
            neighbors_x = interactions[!,"$site,x"][1,1]
            angle_diff_x = config[site].-config[neighbors_x]
            cos_contributions_x += sum(0.5*cos.(angle_diff_x))
            sin_contributions_x += sum(0.5*sin.(angle_diff_x))
        end
        avg_ps_over_rows += ((1/N) * cos_contributions_x) - ((beta/N) * sin_contributions_x^2)
    end
    
    avg_ps_over_cols = 0
    for col in 1:L
        sites_in_cols = cols[!,"col$col"]
        cos_contributions_y = 0
        sin_contributions_y = 0
        for site in sites_in_cols
            neighbors_y = interactions[!,"$site,y"][1,1]
            angle_diff_y = config[site].-config[neighbors_y]
            cos_contributions_y += sum(0.5*cos.(angle_diff_y))
            sin_contributions_y += sum(0.5*sin.(angle_diff_y))
        end
        avg_ps_over_cols += ((1/N) * cos_contributions_y) - ((beta/N) * sin_contributions_y^2)
    end
    
    spin_stiffness = (avg_ps_over_rows+avg_ps_over_cols)/2
            
    return spin_stiffness, avg_ps_over_rows, avg_ps_over_cols
end

In [ ]:
function MonteCarlo_XY(beta,Lx,Ly,warmup_steps,steps,build_lattice_function,rows_and_cols_function,update_function,energy_function,spin_stiffness_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    rows,cols = rows_and_cols_function(Lx,Ly)
    initial_config = rand(Float64,N)*2*pi
    
    # initialize dataframes
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:pss => Float64[],:avpss => Float64[],:accept_rate => Float64[])
    configs = DataFrame()
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    configs[!,:initial] = config
    energy = energy_function(config,interactions)
    for step in (1:warmup_steps)
        config, deltaE, _ = update_function(config,energy,interactions,beta,energy_function)
        energy = energy+deltaE
    end
    configs[!,:after_warmup] = config
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumps = 0
    accept_count = 0
    for step in (1:steps)
        new_config,deltaE,delta_ac = update_function(config,energy,interactions,beta,energy_function)
        config = new_config
        new_energy = energy+deltaE
        energy = new_energy
        accept_count += delta_ac
        
        # calculations
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        ps,_,_ = spin_stiffness_function(beta,config,interactions,rows,cols)
        sumps += ps
        push!(maindf, (sumE/(step*N),sumE2/step,ps,sumps/step,accept_count/(step)))
    end
    configs[!,:final] = config
       
    return maindf, configs
end

In [ ]:
MonteCarlo_XY(100,4,4,500,1000,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness)


# Part 2 Runs

In [ ]:
Ts = collect(0.2:0.01:2.0)
Ls = [8,10,12,14,16]
N_chains = 10
n_steps = 1000
warmup = 500

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        T = round(T,digits=3)
        println("T = ",T," and beta = ",beta)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,configs = MonteCarlo_XY(beta,L,L,warmup,n_steps,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness);
            end
            CSV.write("./data/q2_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q2_cluster/L_$(L)/configs_T_$(T)_chain$(chain).csv",  configs, writeheader=true)
        end
    end
end

In [ ]:
Ts = collect(0.2:0.01:2.0)
Ls = [20,24]
N_chains = 10
n_steps = 1000
warmup = 500

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        T = round(T,digits=3)
        println("T = ",T," and beta = ",beta)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,configs = MonteCarlo_XY(beta,L,L,warmup,n_steps,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness);
            end
            CSV.write("./data/q2_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
#             CSV.write("./data/q2_cluster/L_$(L)/configs_T_$(T)_chain$(chain).csv",  configs, writeheader=true)
        end
    end
end